In [1]:
import numpy as np
from astropy.io import ascii
from astropy.table import Table
import sys
import matplotlib.pyplot as plt
from astropy.cosmology import Planck15 as cosmo


In [2]:
def SyntheticPhotom(wave,flux,response_file,zeropoint=3631.):
    '''Calculates synthetic photom on an input spectrum.
    
    Parameters:
    wave (angstrom): wavelengths of input spectrum in Angstrom
    flux (): flux of spectrum erg/s/cm^2/Hz
    response file: file with two columns with wave/response of filter. 
        Assumes mosfit format (col1 = wave in Ang; col2= transmission).
    zeropoint: zeropoint calibration for the magnitude. Assume Jy default to AB.
    '''
    #read in the response file:
    response = ascii.read(response_file)
    
    #Interpolate response to save wavelengths as flux::
    response2 = np.interp(wave,response['col1'],response['col2'],left=0.0,right=0.0)
    
    #Print a warning if the spectrum does not extend to cover the entire response:
    if (wave[-1] < response['col1'][-1]):
        print('Not enough red spectral coverage for ',response_file)
    if (wave[0] > response['col1'][0]):
        print('Not enough blue spectral coverage for ',response_file)
    
    #Convert wavelengths to Hz:
    c = 2.98e18
    freq = c/wave
    
    #Convert flux to ergs/s/cm^2/Hz (NB: now assumes already in this format):
    flux_Hz = flux 
    
    #Do the synthetic photometry:
    numerator = flux_Hz * response2 / freq
    demoninator = response2 /freq
    flux_int = np.trapz(np.flip(numerator),x=np.flip(freq))/np.trapz(np.flip(demoninator),x=np.flip(freq))
    
    #Calculate the magnitude
    mag = -2.5*np.log10(flux_int/(zeropoint*1e-23))
    return mag


def shift_dist(wave,flux,dist1,z1,dist2=1e-5,z2=0):
    '''Will shift a spectrum to a new distance:
    
    Parameters:
    wave (angstrom): wavelengths of input spectrum in Angstrom
    flux (): flux of spectrum erg/s/cm^2/Ang
    dist1: what is currently assumed (Mpc)
    z1: current redshift
    dist2: what is desired (Mpc). default 10 pc.
    z2: desired redshift (default 0)
    '''

    wave_new = wave * (1+z2)/(1+z1)
    flux_new = flux * (4* np.pi * (dist1*3.086e+24)**2)/(4* np.pi * (dist2*3.086e+24)**2)
    
    return wave_new,flux_new

def flux_from_mag(mag,dmag,zeropoint):
    flux_Jy = zeropoint*10.**(-0.4*mag)
    
    dflux_Jy = np.zeros(len(flux_Jy))
    for i in range(len(dflux_Jy)):
        dflux_Jy[i] = np.sqrt((flux_Jy[i]*np.log(10)*(-0.4)*dmag[i])**2)
    
    return flux_Jy,dflux_Jy

## Define some useful parameters for photometric bands of Interest

In [3]:
#Photometry zeropoints: 
#######################
#PS1 (AB mag)
gp1_zpt = 3631.
rp1_zpt = 3631.
ip1_zpt = 3631.
zp1_zpt = 3631.
#SDSS (AB mag)
us_zpt = 3631.
gs_zpt = 3631.
rs_zpt = 3631.
is_zpt = 3631.
zs_zpt = 3631.
#Generic Bessel (Vega Mag)
U_zpt = 1699.71
B_zpt = 4269.
V_zpt = 3640.
R_zpt = 3056.93
I_zpt = 2550.
#2MASS (Vega Mag)
J_zpt = 1594.
H_zpt = 1024.
K_zpt = 666.7

#######################
##Central Wavelengths:
#####################
#PS1
gp1_pivot = 4876.7
rp1_pivot = 6200.1
ip1_pivot = 7520.8
zp1_pivot = 8665.3

#SDSS
us_pivot = 3586.8
gs_pivot = 4716.7
rs_pivot = 6165.1
is_pivot = 7475.9
zs_pivot = 8922.9

#2MASS:
J_pivot = 12350.
H_pivot = 16620.
K_pivot = 21590.

## PanSTARRS1 Example:

In [41]:
#Some generic information about your object: 
redshift = 0.36
dredshift= 0.11
dist = cosmo.luminosity_distance(redshift).value 
griz = np.array([21.649,20.6994,20.3958,20.0019])
d_griz = np.array([0.1,0.1,0.1,0.1])

#Make a fack spectrum by just interpolating between datapoints: 
zpt_griz = np.array([gp1_zpt,rp1_zpt,ip1_zpt,zp1_zpt])
wave_griz = np.array([gp1_pivot,rp1_pivot,ip1_pivot,zp1_pivot])
flux_Jy,dflux_Jy = flux_from_mag(griz,d_griz,zpt_griz)

wave_test = np.linspace(3900,9510,num=1000)
flux_interp = np.interp(wave_test,wave_griz,flux_Jy)*1e-23

#See how well we reproduce the 'known values' and if you are happy or not.
g_test = SyntheticPhotom(wave_test,flux_interp,'Filters/PAN-STARRS_PS1.g_AB.dat')
r_test = SyntheticPhotom(wave_test,flux_interp,'Filters/PAN-STARRS_PS1.r_AB.dat')
i_test = SyntheticPhotom(wave_test,flux_interp,'Filters/PAN-STARRS_PS1.i_AB.dat')
z_test = SyntheticPhotom(wave_test,flux_interp,'Filters/PAN-STARRS_PS1.z_AB.dat')
print('known mags g {:.2f} r {:.2f} i {:.2f} z {:.2f}'.format(griz[0],griz[1],griz[2],griz[3]))
print('test mags g {:.2f} r {:.2f} i {:.2f} z {:.2f}'.format(g_test,r_test,i_test,z_test))

#Now try to go through and move this spectrum to rest frame and do U and R photometry:
rest_wave,rest_flux = shift_dist(wave_test,flux_interp,dist,redshift)

#And finally, do synthetic photometry on that data in U and R band:
U_abs = SyntheticPhotom(rest_wave,rest_flux,'Filters/Generic_Bessell.U_Vega.dat',zeropoint=U_zpt)
R_abs = SyntheticPhotom(rest_wave,rest_flux,'Filters/Generic_Bessell.R_Vega.dat',zeropoint=R_zpt)
print('Mu = {:.2f} Mr = {:.2f}'.format(U_abs,R_abs))

known mags g 21.65 r 20.70 i 20.40 z 20.00
test mags g 21.49 r 20.74 i 20.36 z 20.04
Not enough red spectral coverage for  Filters/Generic_Bessell.R_Vega.dat
Mu = -20.80 Mr = -21.57


### SDSS Example: 

In [4]:
#Some generic information about your object: 
redshift = 0.107
dredshift= 0.005
dist = cosmo.luminosity_distance(redshift).value 
ugriz = np.array([19.688,18.8809,17.937,17.4958,17.3136])
d_ugriz = np.array([0.1,0.1,0.1,0.1,0.1])

#Make a fack spectrum by just interpolating between datapoints: 
zpt_ugriz = np.array([us_zpt,gs_zpt,rs_zpt,is_zpt,zs_zpt])
wave_ugriz = np.array([us_pivot,gs_pivot,rs_pivot,is_pivot,zs_pivot])
flux_Jy,dflux_Jy = flux_from_mag(ugriz,d_ugriz,zpt_ugriz)

wave_test = np.linspace(2980,11205.0,num=1500)
flux_interp = np.interp(wave_test,wave_ugriz,flux_Jy)*1e-23

#See how well we reproduce the 'known values' and if you are happy or not.
u_test = SyntheticPhotom(wave_test,flux_interp,'Filters/SLOAN_SDSS.u_AB.dat')
g_test = SyntheticPhotom(wave_test,flux_interp,'Filters/SLOAN_SDSS.g_AB.dat')
r_test = SyntheticPhotom(wave_test,flux_interp,'Filters/SLOAN_SDSS.r_AB.dat')
i_test = SyntheticPhotom(wave_test,flux_interp,'Filters/SLOAN_SDSS.i_AB.dat')
z_test = SyntheticPhotom(wave_test,flux_interp,'Filters/SLOAN_SDSS.z_AB.dat')
print('known mags u {:.2f} g {:.2f} r {:.2f} i {:.2f} z {:.2f}'.format(ugriz[0],ugriz[1],ugriz[2],ugriz[3],ugriz[4]))
print('test mags u {:.2f} g {:.2f} r {:.2f} i {:.2f} z {:.2f}'.format(u_test,g_test,r_test,i_test,z_test))

#Now try to go through and move this spectrum to rest frame and do U and R photometry:
rest_wave,rest_flux = shift_dist(wave_test,flux_interp,dist,redshift)

#And finally, do synthetic photometry on that data in U and R band:
U_abs = SyntheticPhotom(rest_wave,rest_flux,'Filters/Generic_Bessell.U_Vega.dat',zeropoint=U_zpt)
R_abs = SyntheticPhotom(rest_wave,rest_flux,'Filters/Generic_Bessell.R_Vega.dat',zeropoint=R_zpt)
print('Mu = {:.2f} Mr = {:.2f}'.format(U_abs,R_abs))

FileNotFoundError: [Errno 2] No such file or directory: 'Filters/SLOAN_SDSS.u_AB.dat'